In [137]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [138]:
print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score,accuracy_score,recall_score

import sys

import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(Fore.YELLOW,Back.BLUE,'V E R I F I C A T I O N S',Style.RESET_ALL)

print('')



Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  V E R I F I C A T I O N S 



### DONNEES DE DEPART
- DEEP LEARNING effectué jusqu'au 31 décembre 2017 (LaDate)
- On attribue un nom au backtest (verif)
- On choisit un close ajusté (adjusted = 'oui')

In [139]:
LaDate = '2017-12-31'
LeNom = 'verif'
adjusted = 'oui'

- Le Scraping comment le 31 décembre 2013
- Le Scraping se termine le 31 décmebre 2018

In [140]:
start = LaDate[:2]+str(int(LaDate[2:4]) - 4)+'-'+LaDate[5:]
start

'2013-12-31'

In [141]:
end = LaDate[:2]+str(int(LaDate[2:4]) + 1)+'-'+LaDate[5:]
end

'2018-12-31'

- L'empreinte (stamp) sera pour le moment au 31 décembre 2018
- Ce curseur variera à chaque fois lors du backtest et commencera à la première date valide de janvier 2018 pour un BT sur 2018

In [142]:
stamp = LaDate[:2]+str(int(LaDate[2:4]) + 1)+'-'+LaDate[5:]
stamp

'2018-12-31'

- Règles d'attribution de Adjusted/Close
- Règles d'attribution du chemin de dossier et du nom de fichier de base

/!\ On devra déplacer l'attribution de LeNom dans la boucle idoine => Attention aussi au scrap des tickers qui utilise ces données

In [143]:
if adjusted == 'oui' or adjusted == 'Oui' or adjusted == 'O' or adjusted == 'o' or adjusted == 'Y' or adjusted == 'y' or adjusted == 'Yes' or adjusted == 'yes':
    print(Fore.GREEN,'On travaille sur Adjusted',Style.RESET_ALL)
    adjusted = 1
    LeNom = LeNom+'_ADJUSTED/'
    LeChemin = 'ADJUSTED/DL_'+stamp+'/'
elif adjusted == 'non' or adjusted == 'Non' or adjusted == 'N' or adjusted == 'n' or adjusted == 'no' or adjusted == 'No':
    print(Fore.GREEN,'On travaille sur Close',Style.RESET_ALL)
    adjusted = 0
    LeNom = LeNom+'_CLOSE/'
    LeChemin = 'CLOSE/DL_'+stamp+'/'
else:
    print(Fore.GREEN,'Ca dépend, ça dépasse. Les calculs sont pas bon Kevin. On passe en ajusted par défaut',Style.RESET_ALL)
    adjusted = 1
    LeNom = LeNom+'_ADJUSTED/'
    LeChemin = 'ADJUSTED/DL_'+stamp+'/'

choupinet_le_canard_il_est_un_vrai_cafard = LeChemin

try:
    os.mkdir('RESULTS')
except:
    pass
try:
    os.mkdir('RESULTS/'+LeNom)
except:
    pass

print('adjusted = ',adjusted,', LeNom = ', LeNom,', LeChemin = ',LeChemin)

 On travaille sur Adjusted 
adjusted =  1 , LeNom =  verif_ADJUSTED/ , LeChemin =  ADJUSTED/DL_2018-12-31/


- On initialise les listes à vide
- On charge constituents, la liste des tickers
- Vérification qu'il n'y a pas de colonne Unnamed: 0
- Vérification qu'il n'y a pas de ticker en doublon

In [144]:
###################
###### MAIN  ######
###################
DATE = []
TICKER = []
PRICE = []
GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE = []

print('Sraping tickers')
constituents = pd.read_csv('New.csv')
#constituents = pd.read_csv(LeChemin+'compteur'+stamp+'.csv')
try:
    constituents = constituents.drop(['Unnamed: 0'],axis=1)
except:
    pass
constituents.head(0)

Sraping tickers


Empty DataFrame
Columns: [Symbol, Name, Sector]
Index: []

In [145]:
print('shape avant drop_duplicate: ',constituents.shape)
constituents = constituents.drop_duplicates(['Symbol'])
#constituents = constituents.drop_duplicates(['Symb.'])
print('shape après déclonage: ',constituents.shape)    
print('Scrap -----> ok')

#today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
#startday = str(dt.date.today()-dt.timedelta(1190))[:5]+str(dt.date.today()-dt.timedelta(1190))[5:7]+str(dt.date.today()-dt.timedelta(1190))[7:10]
#yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
#stamp = input("Enter the date (YYYY-MM-DD): ")

print('Date et heure actuelle : ',time.strftime("%d/%m/%Y %H:%M:%S", time.localtime()))
print('La date scrapée est : ', stamp)

shape avant drop_duplicate:  (505, 3)
shape après déclonage:  (505, 3)
Scrap -----> ok
Date et heure actuelle :  04/11/2019 00:31:55
La date scrapée est :  2018-12-31


#### GENRATION DE LA BASE

- Boucle sur [long = len(constituents)] : Le nombre de ticker, pour parcourir ticker par ticker
    - Pour un ticker donné, on alimente df de start à end
    - On a donc un ticker, par un ticker, les candles du 31 décembre 2013 au 31 décembre 2018
    - On double l'index (date) dans une colonne df['Date']


In [10]:
# def generbaz():
print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
tmps55=time.time()
LaBase = pd.DataFrame()
df = pd.DataFrame()
BADTICKER = []
long = len(constituents)
print('Longueur de long : ',long,' - Date de début : ',start,' - Date de fin : ',end) 

 ON GENERE LaBase 
Longueur de long :  505  - Date de début :  2013-12-31  - Date de fin :  2018-12-31


In [12]:
for loop in range(0, long):
#for loop in range(0, 2):
    if loop == 1:
        print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == round(long/9):
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == round(long/9)*2:
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == round(long/9)*3:
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == round(long/9)*4:
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == round(long/9)*5:
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == round(long/9)*6:
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == round(long/9)*7:
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == round(long/9)*8:
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',end='',flush=True)

    ticker = (constituents.iloc[loop]['Symbol'])
    #ticker = (constituents.iloc[loop]['Symb.'])
    try:
        df = web.DataReader(ticker,'yahoo',start,end)
        df['Date'] = str(df.index[0])[:10]
    except:
        BADTICKER.append(ticker)
        continue

    if adjusted == 1:

        df = df.drop(['Close'],axis=1)
        df['Close'] = df['Adj Close']
        df = df.drop(['Adj Close'],axis = 1)
        #df = df.iloc[-820:,:]


    elif adjusted == 0:
        df = df.drop(['Adj Close'], axis=1)
        df = df[['High', 'Low', 'Open', 'Volume', 'Close']]
        #df = df.iloc[-820:,:]


    df['Symbol'] = ticker
    df = df[['Symbol','High','Low','Open','Volume','Close']]


    LaBase = pd.concat((LaBase,df),ignore_index=False)
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

   |==================  O  ==| 
 Time for completing LaBase Generation = 750.340000 seconds 
 


In [13]:
print('Il y a ',len(BADTICKER),' mauvais tickers \n')
print('Les voici : ')
print(BADTICKER)

Il y a  6  mauvais tickers 

Les voici : 
['BF.B', 'BRK.B', 'CTVA', 'DOW', 'FOX', 'FOXA']


In [14]:
LaBase.to_csv('LABASE_VERFIF_TEST.csv')

In [15]:
LaBase.head(1)

Symbol       High        Low       Open     Volume      Close
Date                                                                    
2013-12-31      A  41.165951  40.829758  41.094421  1316000.0  38.698135

In [149]:
LaBase = pd.read_csv('LABASE_VERFIF_TEST.csv')
LaBase.index = LaBase['Date']
LaBase = LaBase.drop(['Date'],axis=1)

In [150]:
LaBase.tail(1)

Symbol       High        Low       Open     Volume      Close
Date                                                                    
2018-12-31    ZTS  85.589996  84.599998  85.269997  1485200.0  84.998329

- Sauvegarde LaBase_2018-12-31.csv dans le dossier RESULTS/verif_ADJUSTED/

In [17]:
print('Base générée : ',LaBase.shape)
print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
LaBase = LaBase.drop_duplicates()
print('\n \n Base déclonée : ',LaBase.shape)
try:
    os.remove('RESULTS/'+LeNom+'LaBase_'+stamp+'.csv')
except:
    pass
LaBase.to_csv('RESULTS/'+LeNom+'LaBase_'+stamp+'.csv')
    #return(LaBase)

Base générée :  (623913, 6)
Il y a  (949, 6)  Duplicats

 
 Base déclonée :  (622964, 6)


In [151]:
LaBase[LaBase.index >= '2018-01-01 00:00:00']

Symbol       High        Low       Open     Volume      Close
Date                                                                    
2018-01-02      A  67.889999  67.339996  67.419998  1047800.0  66.552505
2018-01-03      A  69.489998  67.599998  67.620003  1698900.0  68.245850
2018-01-04      A  69.820000  68.779999  69.540001  2230700.0  67.733910
2018-01-05      A  70.099998  68.730003  68.730003  1632500.0  68.816872
2018-01-08      A  70.330002  69.550003  69.730003  1613400.0  68.964546
...           ...        ...        ...        ...        ...        ...
2018-12-24    ZTS  80.910004  78.900002  80.910004  1551400.0  78.777962
2018-12-26    ZTS  82.940002  79.139999  79.610001  1869700.0  82.394913
2018-12-27    ZTS  84.330002  81.180000  81.830002  2244700.0  83.795990
2018-12-28    ZTS  85.589996  83.550003  84.830002  1797300.0  83.954971
2018-12-31    ZTS  85.589996  84.599998  85.269997  1485200.0  84.998329

[125249 rows x 6 columns]

### GENERATION DES SIGNAUX => A INTEGRER DANS LA BONNE BOUCLE POUR ECONOMISER DU TEMPS

- On va définir une base sur la période de test
    - On boucle sur chaque ticker pour faire le test
        - On passe chaque date pour 
            - Préparer les données
            - Appliquer les données
            

In [152]:
LaBase[LaBase.index >= '2018-01-01 00:00:00'].index[0] # DEBUT DU TEST

'2018-01-02'

In [153]:
LaBase[LaBase.index >= '2018-01-01 00:00:00'].index[-1] # FIN DU TEST

'2018-12-31'

In [155]:
long = len(constituents)
long # LONG EST TOUJOURS INDEXE SUR LE len(constituents)

505

- On définit stamp comme étant le jour J du live où on établit la génération des signaux

In [156]:
PeriodTest = LaBase[LaBase.index >= '2018-01-01 00:00:00'].index.unique()
PeriodTest

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-16',
       ...
       '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20', '2018-12-21',
       '2018-12-24', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-31'],
      dtype='object', name='Date', length=251)

- On vérifie qu'il y a bien entre 250 et 252 dates sur l'année

In [157]:
for foo,bar in enumerate(sorted(PeriodTest)):
    pass
print('foo : ',foo,' et bar : ',bar)

foo :  250  et bar :  2018-12-31


In [158]:
len(PeriodTest) # 251 dates trouvées pour 2018

251

- On convertit la bar (date en Timestamp pandas) en string pour l'attribuer à stamp

In [159]:
type(bar)

str

In [160]:
str(bar)[:10]

'2018-12-31'

In [161]:
stamp = str(bar)[:10]
stamp

'2018-12-31'

- Pour un ticker donné (ici ZBH) on définit df comme étant LaBase ne concernant que le ticker

In [162]:
ticker = 'AAL'
ticker

'AAL'

In [163]:
df = LaBase[LaBase['Symbol']==ticker]
df.head(1)

Symbol   High        Low   Open     Volume      Close
Date                                                            
2013-12-31    AAL  25.25  24.629999  24.74  7166600.0  23.969639

In [164]:
df.tail(1)

Symbol       High    Low   Open     Volume      Close
Date                                                            
2018-12-31    AAL  32.619999  31.68  32.09  5334700.0  31.818626

In [165]:
df.shape # 1259 dates soit 5 années de 252 jours : 2013 à 2019

(1259, 6)

In [166]:
stamp = '2018-03-19' # ON TESTE SUR UNE DATE AU HASARD DANS LA ZONE DE TEST

In [167]:
len(df[df.index<=stamp]),len(df),stamp,(len(df) - len(df[df.index<=stamp])),df.index[-(len(df) - len(df[df.index<=stamp]))]

(1061, 1259, '2018-03-19', 198, '2018-03-20')

In [168]:
index_fin = -(len(df) - len(df[df.index<=stamp]))
index_fin

-198

- On sélectionne dans cette base de 820 jours avant stamp, jusqu'à stamp

In [169]:
df = df.iloc[index_fin-820:index_fin,:]

In [170]:
df = df.drop(['Symbol'],axis=1) # ON ENLEVE LA COLONNE 'Symbol'

In [171]:
df.shape

(820, 5)

In [172]:
df.head(1)

High    Low       Open      Volume      Close
Date                                                      
2014-12-15  51.32  49.32  50.209999  15368500.0  48.505821

In [173]:
df.tail(1)

High        Low       Open     Volume     Close
Date                                                            
2018-03-19  55.689999  54.189999  55.330002  3834300.0  53.70929

### PREPARATION DES DONNEES

In [174]:
rsi = talib.RSI(df['Close'],timeperiod=14)
stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
sma5 = talib.SMA(df['Close'],timeperiod=5)
sma8 = talib.SMA(df['Close'],timeperiod=8)
sma10 = talib.SMA(df['Close'],timeperiod=10)
sma12 = talib.SMA(df['Close'],timeperiod=12)
sma15 = talib.SMA(df['Close'],timeperiod=15)
sma30 = talib.SMA(df['Close'],timeperiod=30)
sma35 = talib.SMA(df['Close'],timeperiod=35)
sma40 = talib.SMA(df['Close'],timeperiod=40)
sma45 = talib.SMA(df['Close'],timeperiod=45)
sma50 = talib.SMA(df['Close'],timeperiod=50)
atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
delta5_8 = sma5 - sma8
delta8_10 = sma8 - sma10
delta10_12 = sma10 - sma12
delta12_15 = sma12 - sma15
delta15_30 = sma15 - sma30
delta30_35 = sma30 - sma35
delta35_40 = sma35 - sma40
delta40_45 = sma40 - sma45
delta45_50 = sma45 - sma50
bbdelta = upper - middle
price_bolup = df['Close'] - lower
price_bolow = df['Close'] - upper
Ema = talib.EMA(df['Close'],timeperiod=20)
KC_High = Ema + 2*atr
KC_Low = Ema - 2*atr
aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
aroon = aroonup - aroondown 
rsi30_list = []
rsi70_list = []
for i in range(0,df.shape[0]):
    rsi70_list.append(70 - rsi[i])
    rsi30_list.append(rsi[i] - 30)

varop_spy = df['Open'] - df['Close']
varhl_spy = df['High'] - df['Low']
df['Varop_Spy'] = varop_spy
df['Varhl_spy'] = varhl_spy
df['RSI'] = rsi
df['70 - RSI'] = np.array(rsi70_list)
df['RSI - 30'] = np.array(rsi30_list)
df['BBD_Delta_Up'] = bbdelta
df['delta5_8'] = delta5_8
df['delta8_10'] = delta8_10
df['delta10_12'] = delta10_12
df['delta12_15'] = delta12_15
df['delta15_30'] = delta15_30
df['delta30_35'] = delta30_35
df['delta35_40'] = delta35_40
df['delta40_45'] = delta40_45
df['delta45_50'] = delta45_50
df['Stoc_Slowk'] = stoc_slowk
df['Stoc_Slowd'] = stoc_slowd
df['KC_High'] = KC_High
df['KC_Low'] = KC_Low
df['upper'] = upper
df['lower'] = lower
df['var_bollup_kchigh'] = upper-KC_High
df['var_bolllow_kclow'] = lower-KC_Low
df['Aroon Up'] = aroonup
df['Aroon Down'] = aroondown
df['Delta Aroon'] = aroon
up = []
down = []
df = df.dropna()

In [175]:
df.head(1)

High    Low       Open     Volume      Close  Varop_Spy  \
Date                                                                       
2015-02-26  50.380001  49.02  50.380001  7577300.0  47.179611    3.20039   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...  Stoc_Slowd  \
Date                                                    ...               
2015-02-26   1.360001  47.684392  22.315608  17.684392  ...     2.38402   

              KC_High    KC_Low      upper      lower  var_bollup_kchigh  \
Date                                                                       
2015-02-26  53.658902  41.57623  49.874051  45.392332          -3.784851   

            var_bolllow_kclow   Aroon Up  Aroon Down  Delta Aroon  
Date                                                               
2015-02-26           3.816101  66.666667         0.0    66.666667  

[1 rows x 31 columns]

In [176]:
df.tail(1)

High        Low       Open     Volume     Close  Varop_Spy  \
Date                                                                          
2018-03-19  55.689999  54.189999  55.330002  3834300.0  53.70929   1.620712   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...  Stoc_Slowd  \
Date                                                    ...               
2018-03-19        1.5  53.347543  16.652457  23.347543  ...   -4.199413   

              KC_High     KC_Low      upper      lower  var_bollup_kchigh  \
Date                                                                        
2018-03-19  57.134116  49.809694  55.900889  52.957717          -1.233227   

            var_bolllow_kclow   Aroon Up  Aroon Down  Delta Aroon  
Date                                                               
2018-03-19           3.148023  55.555556         0.0    55.555556  

[1 rows x 31 columns]

In [177]:
df.shape

(771, 31)

In [178]:
df.isnull().sum().sum()

0

### BOOSTING

In [179]:
X = df.copy() # ON SECURISE df EN CLONANT DANS X
X.head()

High        Low       Open      Volume      Close  Varop_Spy  \
Date                                                                            
2015-02-26  50.380001  49.020000  50.380001   7577300.0  47.179611   3.200390   
2015-02-27  49.400002  47.849998  49.000000   9396400.0  45.793385   3.206615   
2015-03-02  48.570000  47.849998  48.200001  11482900.0  46.185364   2.014637   
2015-03-03  49.029999  47.750000  47.900002   9051200.0  46.625126   1.274876   
2015-03-04  48.959999  47.980000  48.849998   6711000.0  46.797207   2.052792   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...  Stoc_Slowd  \
Date                                                    ...               
2015-02-26   1.360001  47.684392  22.315608  17.684392  ...    2.384020   
2015-02-27   1.550003  42.599460  27.400540  12.599460  ...  -26.353059   
2015-03-02   0.720001  44.404803  25.595197  14.404803  ...  -43.302295   
2015-03-03   1.279999  46.440077  23.559923  16.440077  ...  -47.677189   
2015-03-04   0.980000  47.253801  22.746199  17.253801  ...  -42.602087   

              KC_High     KC_Low      upper      lower  var_bollup_kchigh  \
Date                                                                        
2015-02-26  53.658902  41.576230  49.874051  45.392332          -3.784851   
2015-02-27  53.325115  41.562554  49.964244  45.198039          -3.360870   
2015-03-02  53.172455  41.475505  49.911422  45.314600          -3.261033   
2015-03-03  53.089977  41.424868  49.923014  45.188287          -3.166964   
2015-03-04  52.929866  41.497319  49.881698  45.053270          -3.048168   

            var_bolllow_kclow   Aroon Up  Aroon Down  Delta Aroon  
Date                                                               
2015-02-26           3.816101  66.666667    0.000000    66.666667  
2015-02-27           3.635485  55.555556    0.000000    55.555556  
2015-03-02           3.839095  44.444444  100.000000   -55.555556  
2015-03-03           3.763420  33.333333  100.000000   -66.666667  
2015-03-04           3.555952  22.222222   88.888889   -66.666667  

[5 rows x 31 columns]

In [180]:
X = X.drop(['Close'],axis=1) # ON VIRE LA CONLONNE 'Close' DE X

In [181]:
X['Close'] = df['Close'] # ON RECUPERE LE 'CLOSE' DE df ET ON LE POSE GENITLLEMENT A LA FIN DE X

In [182]:
X = X.astype(np.float64) # ON S'ASSURE QUE TOUT EST EN FLOAT64 ET QU'IL NE TRAINE PAS DES 32 OU DES INT

In [183]:
y = X.iloc[:,-1] # ON ATTRIBUE A y LA DERNIERE COLONNE DE X, SOIT LES 'Close'
y.tail(2)

Date
2018-03-16    54.476284
2018-03-19    53.709290
Name: Close, dtype: float64

In [184]:
Xtrain = X.iloc[:-2,:-1] # Xtrain est composé de X moins les 'Close' SAUF les deux dernières valeurs

In [185]:
Xtest = X.iloc[-2:-1,:-1] # Xtrain est composé de X moins les 'Close' MAIS QUE SUR les deux dernières valeurs

In [186]:
yshift = y.shift(-1) # ON REMONTE LES 'Close' D'UNE LIGNE. DONC IL DONNE LE CLOSE DU LENDEMAIN SUR LA LIGNE DE LA VEILLE
yshift.tail(2)

Date
2018-03-16    53.70929
2018-03-19         NaN
Name: Close, dtype: float64

In [187]:
ytrain = yshift.iloc[:-2] # On attribue à ytrain toutes les lignes SAUF LES 2 DERNIERES

In [188]:
ytest = yshift.iloc[-2:-1]
ytest

Date
2018-03-16    53.70929
Name: Close, dtype: float64

In [189]:
model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

In [190]:
model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=1,
       importance_type='gain', learning_rate=1, max_delta_step=0,
       max_depth=100, min_child_weight=1, missing=None, n_estimators=20000,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [191]:
ytrain_pred = model.predict(Xtrain)

In [192]:
y_pred = model.predict(Xtest)

In [193]:
pred = model.predict(X.iloc[:,:-1])

In [194]:
df['Close.S'] = pred
df['Close.S2'] = df['Close.S']
df = df.dropna()

In [195]:
df.head(1)

High    Low       Open     Volume      Close  Varop_Spy  \
Date                                                                       
2015-02-26  50.380001  49.02  50.380001  7577300.0  47.179611    3.20039   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...    KC_Low  \
Date                                                    ...             
2015-02-26   1.360001  47.684392  22.315608  17.684392  ...  41.57623   

                upper      lower  var_bollup_kchigh  var_bolllow_kclow  \
Date                                                                     
2015-02-26  49.874051  45.392332          -3.784851           3.816101   

             Aroon Up  Aroon Down  Delta Aroon    Close.S   Close.S2  
Date                                                                  
2015-02-26  66.666667         0.0    66.666667  45.818848  45.818848  

[1 rows x 33 columns]

In [196]:
df.tail(1)

High        Low       Open     Volume     Close  Varop_Spy  \
Date                                                                          
2018-03-19  55.689999  54.189999  55.330002  3834300.0  53.70929   1.620712   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...     KC_Low  \
Date                                                    ...              
2018-03-19        1.5  53.347543  16.652457  23.347543  ...  49.809694   

                upper      lower  var_bollup_kchigh  var_bolllow_kclow  \
Date                                                                     
2018-03-19  55.900889  52.957717          -1.233227           3.148023   

             Aroon Up  Aroon Down  Delta Aroon    Close.S   Close.S2  
Date                                                                  
2018-03-19  55.555556         0.0    55.555556  52.944263  52.944263  

[1 rows x 33 columns]

### FIN DE PREPARATION DES DONNEES

In [197]:
df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
for i in range(0,df.shape[0]-5):
    if df.iloc[i]['%Futur'] > 0.1 :
        up.append(1)
        down.append(0)
    elif df.iloc[i]['%Futur'] < -0.1:
        up.append(0)
        down.append(1)
    else:
        up.append(0)
        down.append(0)
up.append(0)
down.append(0)
up.append(0)
down.append(0)
up.append(0)
down.append(0)
up.append(0)
down.append(0)
up.append(0)
down.append(0)


df['target_up'] = up  
df['target_down'] = down 

In [198]:
df.head(1)

High    Low       Open     Volume      Close  Varop_Spy  \
Date                                                                       
2015-02-26  50.380001  49.02  50.380001  7577300.0  47.179611    3.20039   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...  \
Date                                                    ...   
2015-02-26   1.360001  47.684392  22.315608  17.684392  ...   

            var_bolllow_kclow   Aroon Up  Aroon Down  Delta Aroon    Close.S  \
Date                                                                           
2015-02-26           3.816101  66.666667         0.0    66.666667  45.818848   

             Close.S2   %Futur  %Futur2  target_up  target_down  
Date                                                             
2015-02-26  45.818848 -2.88422 -2.88422          0            1  

[1 rows x 37 columns]

- On établit les signaux pour des changements repérés à 0.5%. Il y a 55.51% des signaux qui répondent à ce critère

In [199]:
df.target_up.sum(),df.target_down.sum(),df.shape,(df.target_up.sum() + df.target_down.sum())*100/df.shape[0]

(363, 353, (771, 37), 92.86640726329442)

### DEEP LEARNING
    - On charge les modèles de deep learning up et down
    - On les applique

In [200]:
LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]

'ADJUSTED/DL_2017-12-31/'

In [201]:
yamlup = (LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]+'Save_'+ticker+'_up.yaml')
yamldown = (LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]+'Save_'+ticker+'_down.yaml')
modelup = (LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]+'Save_'+ticker+'_up.h5')
modeldown = (LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]+'Save_'+ticker+'_down.h5')

yaml_file_up = open(yamlup, 'r')
yaml_file_down = open(yamldown, 'r')
model_yaml_up = yaml_file_up.read()
model_yaml_down = yaml_file_down.read()
yaml_file_up.close()
yaml_file_down.close()
model_up = model_from_yaml(model_yaml_up)
model_down = model_from_yaml(model_yaml_down)
# load weights into new model
model_up.load_weights(modelup)
model_down.load_weights(modeldown)

In [202]:
X = df.copy() # ON RESERVE df QUE L'ON CLONE POUR LES BESOINS DANS X
X.tail(1)

High        Low       Open     Volume     Close  Varop_Spy  \
Date                                                                          
2018-03-19  55.689999  54.189999  55.330002  3834300.0  53.70929   1.620712   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...  \
Date                                                    ...   
2018-03-19        1.5  53.347543  16.652457  23.347543  ...   

            var_bolllow_kclow   Aroon Up  Aroon Down  Delta Aroon    Close.S  \
Date                                                                           
2018-03-19           3.148023  55.555556         0.0    55.555556  52.944263   

             Close.S2    %Futur   %Futur2  target_up  target_down  
Date                                                               
2018-03-19  52.944263 -1.424383 -1.424383          0            0  

[1 rows x 37 columns]

In [203]:
X = X.iloc[:,:-4] # ON ENLEVE LES 4 DERNIERES COLONNES '%Futur','%Futur2','target_up','target_down'
X.tail(1)

High        Low       Open     Volume     Close  Varop_Spy  \
Date                                                                          
2018-03-19  55.689999  54.189999  55.330002  3834300.0  53.70929   1.620712   

            Varhl_spy        RSI   70 - RSI   RSI - 30  ...     KC_Low  \
Date                                                    ...              
2018-03-19        1.5  53.347543  16.652457  23.347543  ...  49.809694   

                upper      lower  var_bollup_kchigh  var_bolllow_kclow  \
Date                                                                     
2018-03-19  55.900889  52.957717          -1.233227           3.148023   

             Aroon Up  Aroon Down  Delta Aroon    Close.S   Close.S2  
Date                                                                  
2018-03-19  55.555556         0.0    55.555556  52.944263  52.944263  

[1 rows x 33 columns]

In [204]:
X = X.astype(np.float64) # ON S'ASSURE QUE TOUT EST EN FLOAT64 ET QU'IL NE TRAINE PAS DES 32 OU DES INT

In [205]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [206]:
X = scaler.fit_transform(X)

In [207]:
X[-5]

array([0.94911563, 0.94628223, 0.94092439, 0.02206066, 0.93646407,
       0.41556891, 0.19301448, 0.74717788, 0.25282212, 0.74717788,
       0.36123766, 0.73235257, 0.58763057, 0.49186479, 0.62237023,
       0.70908209, 0.41152659, 0.16968772, 0.45450189, 0.59441169,
       0.92237482, 0.91077163, 0.94750968, 0.95065202, 0.8707101 ,
       0.89465956, 0.63132887, 0.49927953, 1.        , 0.22222222,
       0.88888889, 0.92147655, 0.92147655])

In [208]:
Xtest = X[-1:,:]
len(Xtest)

1

In [209]:
seed = 770
np.random.seed(seed)

In [210]:
yhat_up = model_up.predict_classes(Xtest)
yhat_down = model_down.predict_classes(Xtest)

predict_up = model_up.predict(Xtest)
predict_down = model_down.predict(Xtest)

# La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)

In [211]:
resultats = pd.DataFrame()

In [212]:
df.index[-1:]

Index(['2018-03-19'], dtype='object', name='Date')

In [213]:
len(df.index[-1:])

1

In [214]:
df.index[-1:]

Index(['2018-03-19'], dtype='object', name='Date')

In [215]:
resultats['Date'] = df.index[-1:]

In [216]:
resultats.index= df.index[-1:]

In [217]:
resultats['Move Up'] = yhat_up

In [218]:
resultats['Confiance up'] = (predict_up)*100

In [219]:
resultats['Move Down'] = yhat_down

In [220]:
resultats['Confiance Down'] = (predict_down)*100

In [221]:
resultats['Actual'] = df.iloc[-1:]['Close']

In [222]:
resultats['Actual.S'] = df.iloc[-1:]['Close.S']

In [223]:
resultats

Date  Move Up  Confiance up  Move Down  Confiance Down  \
Date                                                                       
2018-03-19  2018-03-19        0       0.00003          1       99.999985   

              Actual   Actual.S  
Date                             
2018-03-19  53.70929  52.944263

In [224]:
resultats['Confiance up'].sum(),resultats['Confiance Down'].sum()

(2.9769646e-05, 99.999985)

In [225]:
open_S = LaBase[LaBase['Symbol']==ticker].iloc[:]['Open'].shift(-1)

In [226]:
open_S.head(1)

Date
2013-12-31    25.07
Name: Open, dtype: float64

In [227]:
open_S.tail(1)

Date
2018-12-31   NaN
Name: Open, dtype: float64

In [228]:
stamp,str(stamp)[:10]

('2018-03-19', '2018-03-19')

In [229]:
open_S[open_S.index == str(stamp)[:10]]

Date
2018-03-19    54.93
Name: Open, dtype: float64

In [230]:
resultats['Open.S'] = open_S[open_S.index == str(stamp)[:10]]
resultats['Open.S']

Date
2018-03-19    54.93
Name: Open.S, dtype: float64

In [231]:
dmp_cp=[]

In [232]:
dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)

In [233]:
resultats['DMP_CP'] = dmp_cp

In [234]:
resultats

Date  Move Up  Confiance up  Move Down  Confiance Down  \
Date                                                                       
2018-03-19  2018-03-19        0       0.00003          1       99.999985   

              Actual   Actual.S  Open.S     DMP_CP  
Date                                                
2018-03-19  53.70929  52.944263   54.93 -99.999939

In [235]:
list_dir = os.listdir(LeChemin+'.')

In [236]:
pikunichihouahoua = 3

In [237]:
str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1]

'CFG'

In [238]:
NEW_LIST = []

In [239]:
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])

In [240]:
len(NEW_LIST)

574

In [241]:
NEW_LIST[0]

'VMC'

In [242]:
NEW_LIST = sorted(NEW_LIST)
NEW_LIST[0]

'A'

In [243]:
NEW_LIST = (set(NEW_LIST))

In [244]:
len(NEW_LIST)

287

In [245]:
LeChemin

'ADJUSTED/DL_2018-12-31/'

AttributeError: 'numpy.int64' object has no attribute 'listdir'

    -> Il ne reste plus qu'à tout intégrer dans les boucles

In [270]:
tmps55=time.time()

delta = round(df.shape[0])
bloc1 = round(delta*0.80)
bloc2 = delta - bloc1

filtre_up = 93
filtre_down = 97
filtre_up_close = 80
filtre_down_close = 80

LeChemin = choupinet_le_canard_il_est_un_vrai_cafard

LeChemin = LeChemin[:14]+str(int(LeChemin[14:16])-1)+LeChemin[16:]

list_dir = os.listdir(LeChemin+'.')

NEW_LIST = []

for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])

NEW_LIST = set(sorted(NEW_LIST))

for foo,bar in enumerate(sorted(PeriodTest)): # ON BOUCLE SUR LA DATE
    
    stamp = str(bar)[:10] # ON ATTRIBUE LE STAMP
    
    print('\n Date : ',stamp)
    
    DDATE = []
    MOVE_UP = []
    CONF_UP = []
    MOVE_DWN = []
    CONF_DWN = []
    ACTUAL = []
    ACTUAL_S = []
    OPEN_S = []
    DMP_CP = []
    
    DATE = []
    TICKER = []
    GREEN = []
    GREEN_CLOSE = []
    RED = []
    RED_CLOSE = []
    PRICE = []
    
    
    for loop,ticker in enumerate(NEW_LIST): # ON BOUCLE SUR LE TICKER
        tmps73=time.time()
        if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
        if loop == round(long/9):
            print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
        if loop == round(long/9)*2:
            print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
        if loop == round(long/9)*3:
            print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
        if loop == round(long/9)*4:
            print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
        if loop == round(long/9)*5:
            print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
        if loop == round(long/9)*6:
            print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
        if loop == round(long/9)*7:
            print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
        if loop == round(long/9)*8:
            print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',end='',flush=True)
            
        
        yamlup = (LeChemin[:14]+str(int(LeChemin[14:16])-0)+LeChemin[16:]+'Save_'+ticker+'_up.yaml')
        yamldown = (LeChemin[:14]+str(int(LeChemin[14:16])-0)+LeChemin[16:]+'Save_'+ticker+'_down.yaml')
        modelup = (LeChemin[:14]+str(int(LeChemin[14:16])-0)+LeChemin[16:]+'Save_'+ticker+'_up.h5')
        modeldown = (LeChemin[:14]+str(int(LeChemin[14:16])-0)+LeChemin[16:]+'Save_'+ticker+'_down.h5')

        yaml_file_up = open(yamlup, 'r')
        yaml_file_down = open(yamldown, 'r')
        model_yaml_up = yaml_file_up.read()
        model_yaml_down = yaml_file_down.read()
        yaml_file_up.close()
        yaml_file_down.close()
        model_up = model_from_yaml(model_yaml_up)
        model_down = model_from_yaml(model_yaml_down)
        # load weights into new model
        model_up.load_weights(modelup)
        model_down.load_weights(modeldown)
        
        
        #ticker = (constituents.iloc[loop]['Symb.']) # ON ATTRIBUE LE TICKER
        
        df = LaBase[LaBase['Symbol']==ticker] # ON REDUIT LaBase A df SUR CE TICKER
        
        if df.shape[0]<820: # Si jamais il n'y a pas assez de valeurs (genre nouveau ticker dans le S&P)
            continue # Alors, change donc de Ticker
        
        index_fin = -(len(df) - len(df[df.index<=stamp])) # ON DEFINIT L'INDEX DE FIN DE TEST A PARTIR DE stamp
        
        df = df.iloc[index_fin-820:index_fin,:] # ON REDUIT df AUX 820 DERNIERES VALEURS JUSQU'A STAMP
        
        df = df.drop(['Symbol'],axis=1) # ON ENLEVE LA COLONNE 'Symbol'
        
        ###############################
        ### PEPARATION DES DONNEES ###
        ##############################
        
        
        rsi = talib.RSI(df['Close'],timeperiod=14)
        stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
        upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
        sma5 = talib.SMA(df['Close'],timeperiod=5)
        sma8 = talib.SMA(df['Close'],timeperiod=8)
        sma10 = talib.SMA(df['Close'],timeperiod=10)
        sma12 = talib.SMA(df['Close'],timeperiod=12)
        sma15 = talib.SMA(df['Close'],timeperiod=15)
        sma30 = talib.SMA(df['Close'],timeperiod=30)
        sma35 = talib.SMA(df['Close'],timeperiod=35)
        sma40 = talib.SMA(df['Close'],timeperiod=40)
        sma45 = talib.SMA(df['Close'],timeperiod=45)
        sma50 = talib.SMA(df['Close'],timeperiod=50)
        atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
        delta5_8 = sma5 - sma8
        delta8_10 = sma8 - sma10
        delta10_12 = sma10 - sma12
        delta12_15 = sma12 - sma15
        delta15_30 = sma15 - sma30
        delta30_35 = sma30 - sma35
        delta35_40 = sma35 - sma40
        delta40_45 = sma40 - sma45
        delta45_50 = sma45 - sma50
        bbdelta = upper - middle
        price_bolup = df['Close'] - lower
        price_bolow = df['Close'] - upper
        Ema = talib.EMA(df['Close'],timeperiod=20)
        KC_High = Ema + 2*atr
        KC_Low = Ema - 2*atr
        aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
        aroon = aroonup - aroondown 
        rsi30_list = []
        rsi70_list = []
        for i in range(0,df.shape[0]):
            rsi70_list.append(70 - rsi[i])
            rsi30_list.append(rsi[i] - 30)

        varop_spy = df['Open'] - df['Close']
        varhl_spy = df['High'] - df['Low']
        df['Varop_Spy'] = varop_spy
        df['Varhl_spy'] = varhl_spy
        df['RSI'] = rsi
        df['70 - RSI'] = np.array(rsi70_list)
        df['RSI - 30'] = np.array(rsi30_list)
        df['BBD_Delta_Up'] = bbdelta
        df['delta5_8'] = delta5_8
        df['delta8_10'] = delta8_10
        df['delta10_12'] = delta10_12
        df['delta12_15'] = delta12_15
        df['delta15_30'] = delta15_30
        df['delta30_35'] = delta30_35
        df['delta35_40'] = delta35_40
        df['delta40_45'] = delta40_45
        df['delta45_50'] = delta45_50
        df['Stoc_Slowk'] = stoc_slowk
        df['Stoc_Slowd'] = stoc_slowd
        df['KC_High'] = KC_High
        df['KC_Low'] = KC_Low
        df['upper'] = upper
        df['lower'] = lower
        df['var_bollup_kchigh'] = upper-KC_High
        df['var_bolllow_kclow'] = lower-KC_Low
        df['Aroon Up'] = aroonup
        df['Aroon Down'] = aroondown
        df['Delta Aroon'] = aroon
        up = []
        down = []
        df = df.dropna()
        
        
        #############
        ### BOOST ###
        #############
        
        X = df.copy() # ON SECURISE df EN CLONANT DANS X
        
        X = X.drop(['Close'],axis=1) # ON VIRE LA CONLONNE 'Close' DE X
        
        X['Close'] = df['Close'] # ON RECUPERE LE 'CLOSE' DE df ET ON LE POSE GENITLLEMENT A LA FIN DE X
        
        X = X.astype(np.float64) # ON S'ASSURE QUE TOUT EST EN FLOAT64 ET QU'IL NE TRAINE PAS DES 32 OU DES INT
        
        y = X.iloc[:,-1] # ON ATTRIBUE A y LA DERNIERE COLONNE DE X, SOIT LES 'Close'
        
        Xtrain = X.iloc[:-2,:-1] # Xtrain est composé de X moins les 'Close' SAUF les deux dernières valeurs
        
        Xtest = X.iloc[-2:-1,:-1] # Xtrain est composé de X moins les 'Close' MAIS QUE SUR les deux dernières valeurs
        
        yshift = y.shift(-1) # ON REMONTE LES 'Close' D'UNE LIGNE. DONC IL DONNE LE CLOSE DU LENDEMAIN SUR LA LIGNE DE LA VEILLE
        
        ytrain = yshift.iloc[:-2]
        
        ytest = yshift.iloc[-2:-1]

        model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

        model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

        ytrain_pred = model.predict(Xtrain)

        y_pred = model.predict(Xtest)

        pred = model.predict(X.iloc[:,:-1])

        df['Close.S'] = pred
        df['Close.S2'] = df['Close.S']
        df = df.dropna()
        
        
        #######################
        ### FIN DE PREPA DF ###
        #######################
        
        df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
        df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
        for i in range(0,df.shape[0]-5):
            if df.iloc[i]['%Futur'] > 0.73 :
                up.append(1)
                down.append(0)
            elif df.iloc[i]['%Futur'] < -0.73:
                up.append(0)
                down.append(1)
            else:
                up.append(0)
                down.append(0)
        up.append(0)
        down.append(0)
        up.append(0)
        down.append(0)
        up.append(0)
        down.append(0)
        up.append(0)
        down.append(0)
        up.append(0)
        down.append(0)


        df['target_up'] = up  
        df['target_down'] = down
        
        
        #####################
        ### DEEP LEARNING ###
        #####################

        X = df.copy()
        X = X.iloc[:,:-4]
        y_up = df.iloc[:,-2].values
        y_down = df.iloc[:,-1].values

        scaler = MinMaxScaler(feature_range=(0, 1))
        X = X.astype(np.float64) # ON S'ASSURE QUE TOUT EST EN FLOAT64 ET QU'IL NE TRAINE PAS DES 32 OU DES INT
        X = scaler.fit_transform(X)
        y_up = np.array(y_up).reshape(-1,1)
        y_down = np.array(y_down).reshape(-1,1)

        Xtrain = X[:bloc1,:]
        Xtest = X[bloc1:,:]
        ytrain_up = y_up[:bloc1,:]
        ytest_up = y_up[bloc1:,:]
        ytrain_down = y_down[:bloc1,:]
        ytest_down = y_down[bloc1:,:]

        seed = 770
        np.random.seed(seed)

        ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
        ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

        Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

        yhat_up = model_up.predict_classes(Xtest)
        yhat_down = model_down.predict_classes(Xtest)

        predict_up = model_up.predict(Xtest)
        predict_down = model_down.predict(Xtest)

        #accuracy_up = accuracy_score(ytest_up, yhat_up)
        #accuracy_down = accuracy_score(ytest_down, yhat_down)

        # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
        precision_up = precision_score(ytest_up, yhat_up)  
        precision_down = precision_score(ytest_down, yhat_down) 


        # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
        #recall_up = recall_score(ytest_up, yhat_up) 
        #recall_down = recall_score(ytest_down, yhat_down) 


        resultats = pd.DataFrame()
        resultats['Date'] = df.index[bloc1:]
        resultats.index= df.index[bloc1:]
        resultats['Move Up'] = yhat_up
        resultats['Confiance up'] = (predict_up)*100
        resultats['Move Down'] = yhat_down
        resultats['Confiance Down'] = (predict_down)*100
        resultats['Actual'] = df.iloc[bloc1:]['Close']
        resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
        open_S = df['Open'].shift(-1)
        resultats['Open.S'] = open_S.iloc[bloc1:]
        dmp_cp=[]
        dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
        resultats['DMP_CP'] = dmp_cp
         
        
        #######################################
        ### PREPARATION RESULTATS = SIGNAUX ###
        #######################################
        
        DDATE.append(str(df.index[-1])[:10])
        MOVE_UP.append(yhat_up[-1][-1])
        CONF_UP.append((predict_up[-1][-1])*100)
        MOVE_DWN.append(yhat_down[-1][-1])
        CONF_DWN.append((predict_down[-1][-1])*100)
        ACTUAL.append(df.iloc[-1]['Close'])
        ACTUAL_S.append(df.iloc[-1]['Close.S'])
        OPEN_S.append(open_S[open_S.index == str(stamp)[:10]][-1])
        DMP_CP.append((((predict_up[-1][-1])*100)-(predict_down[-1][-1])*100)/((predict_up[-1][-1])*100+(predict_down[-1][-1])*100)*100)
        
        i = -1
        
        price = df.iloc[i]['Close']
        
        if (precision_up * 100) > 73 and (precision_down * 100) > 73:

            if resultats.iloc[i]['Confiance Down'] > filtre_down:
                doob = -1
            elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
            elif resultats.iloc[i]['Confiance up'] > filtre_up:
                doob = 1
            elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
            else :
                doob = 0
                waab = 0

            if doob == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            elif doob == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            elif doob == 0 and waab == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            elif doob == 0 and waab == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            else:
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(0)
                PRICE.append(price)


        else:
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(0)
            PRICE.append(price)


        
        
        ########################
        ####### BACKTEST########
        ########################
        pos_long = 0
        pos_short = 0
        pactol = 10000/beta
        price_buy = 0
        price_sell = 0
        x = 0
        winners = []
        loosers = []
        nb_win = 0
        nb_los = 0
        mini_pnl = 0
        pos_duration = []
        average_duration = resultats.iloc[0,0] - resultats.iloc[0,0]
        trades = []
        eq_curx = []
        eq_cury = []
        maxdd = []
        nbS = 0
        nbL = 0

        for i in range(0,len(resultats)-1):
            if pos_long == 0 and pos_short == 0 and resultats.iloc[i]['Green'] == 1 :
                pos_long = 1
                nbL+=1
                x = round((10000/beta)/resultats.iloc[i]['Open.S'])
                price_buy = round(resultats.iloc[i]['Open.S'],2)
                time_enter = resultats.iloc[i]['Date']

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 1,\
                                   'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : 0,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                           'Latent PNL':0}],ignore_index=True)

                print('\n Le ',str(resultats.iloc[i]['Date']),', achat de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
                print('Nouveau pactol : ',round(pactol,2))
                trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],1))

            elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 0 :

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : 0,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                           'Latent PNL':round((resultats.iloc[i]['Open.S'] - price_buy)*x,2)}],ignore_index=True)
                try:
                    maxdd.append(round((resultats.iloc[i]['Open.S'] - price_buy)*x,2))
                except:
                    print('Null val of maxdd')
                    maxdd.append('Null')


            elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 1 :
                old_pactol = round(pactol,2)
                eq_curx.append(resultats.iloc[i]['Date'])
                eq_cury.append(old_pactol)
                pos_long = 0
                price_sell = round(resultats.iloc[i]['Open.S'],2)
                mini_pnl = round(x*(price_sell-price_buy),2)
                trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],2))

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : mini_pnl,\
                                           'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

                pactol = round((pactol + mini_pnl),2)
                print('\n Le ',str(resultats.iloc[i]['Date']),', vente pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
                print('Nouveau pactol : ',pactol)
                print('pnl:', mini_pnl)
                print('prix de vente',price_sell)
                print('prix d achat',price_buy)

                pos_duration.append(resultats.iloc[i]['Date'] - time_enter)

                if mini_pnl > 0:
                    winners.append(mini_pnl)
                    mini_pnl = 0
                    nb_win +=1
                else :
                    loosers.append(mini_pnl)
                    mini_pnl = 0
                    nb_los +=1


            elif pos_short == 0 and pos_long == 0 and resultats.iloc[i]['Red']:
                pos_short = 1
                nbS+=1
                x = round((10000/beta)/resultats.iloc[i]['Open.S'])
                price_sell = round(resultats.iloc[i]['Open.S'],2)
                time_enter = resultats.iloc[i]['Date']

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 0, 'Open Short' : 1, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : 0,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

                print('\n Le ',str(resultats.iloc[i]['Date']),', vente de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
                print('Nouveau pactol : ',pactol)
                trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-1))


            elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 0 :

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : 0,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                           'Latent PNL':-round((resultats.iloc[i]['Open.S'] - price_sell)*x,2)}],ignore_index=True)
                try:
                    maxdd.append(round((resultats.iloc[i]['Open.S'] - price_sell)*x,2))
                except:
                    print('Null val of maxdd')
                    maxdd.append(0)

            elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 1 :
                old_pactol = round(pactol,2)
                eq_curx.append(resultats.iloc[i]['Date'])
                eq_cury.append(old_pactol)
                pos_short = 0
                price_buy = round(resultats.iloc[i]['Open.S'],2)
                trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-2))
                mini_pnl = round(x*(price_sell-price_buy),2)

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                                   'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : mini_pnl,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

                pactol = round((pactol + mini_pnl),2)
                print('\n Le ',str(resultats.iloc[i]['Date']),', achat pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
                print('Nouveau pactol : ',pactol)

                print('prix de vente',price_sell)
                print('prix d achat',price_buy)

                print('pnl:', mini_pnl)
                pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
                if mini_pnl > 0:
                    winners.append(mini_pnl)
                    mini_pnl = 0
                    nb_win +=1
                else :
                    loosers.append(mini_pnl)
                    mini_pnl = 0
                    nb_los +=1
            else:

                signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : 0, 'Quantity' : 0, 'PNL' : 0,\
                                           'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

        if pos_long == 1:
            pos_long = 0
            print('Cutting non resolved position')
            print(' Le price buy était de : ',price_buy)
            x = round((10000/beta)/resultats.iloc[i]['Open.S'])
            print('x = ',x)
            price_sell = round(resultats.iloc[i]['Open.S'],2)
            print('Et le price sell : ',price_sell)
            mini_pnl = round(x*(price_sell-price_buy),2)

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                                   'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : mini_pnl,\
                                       'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            print('pnl : ',mini_pnl)
            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1


        if pos_short == 1:
            pos_short = 0
            print('Cutting non resolved position')
            print(' Le price sell était de : ',price_sell)
            x = round((10000/beta)/resultats.iloc[i]['Open.S'])
            print('x = ',x)
            price_buy = round(resultats.iloc[i]['Open.S'],2)
            print('Et le price_buy : ',price_buy)
            mini_pnl = round(x*(price_sell-price_buy),2)

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                                   'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                                   'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : 0,\
                                       'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':mini_pnl}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            print('pnl : ',mini_pnl)

            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1


        pnl = round(pactol - (10000/beta),2)
        print('\n Begining of BackTest :',resultats.iloc[0]['Date'])
        print('Instrument :',ticker)
        print('Face value per trade : $',10000/beta)
        print('End of BackTest :',resultats.iloc[-1]['Date'])
        duration = pd.to_timedelta((resultats.iloc[-1]['Date'] - resultats.iloc[0]['Date']),unit='d')
        print("BackTest's period :",duration)
        print('Pnl :',round(pnl,2),'$')
        print('% Pnl : ',round(pnl/(10000/beta)*100,2))
        print('Total winners :',round(sum(winners),2),'$')
        print('Total loosers :',round(sum(loosers),2),'$')
        print('Number of winners :',round(nb_win))
        print('Number of loosers :',round(nb_los))
        print('Maximum Drawdown : ',round(min(maxdd),2))
        print('% Max Drawdown : ',round(min(maxdd)/(10000/beta)*100,2),'%')
        print('Nombre toatl de trades : ',round(nbL+nbS,2))
        print('% de trades Long : ',round((nbL*100)/(nbL+nbS),2))
        print('% de trades Short : ',round((nbS*100)/(nbL+nbS),2))
        try:
            print('% winners :', round((nb_win/(nb_win+nb_los) * 100),2),'%')
            print('Average winners :',(round(sum(winners)/(nb_win))))
        except:
            print('% winners : null')
        try:
            print('Average losers :',(round(sum(loosers)/(nb_los))))
        except:
            print('Average losers : Inf')
        try:
            for t in range(0,len(pos_duration)):
                average_duration = pd.to_timedelta((average_duration + pos_duration[t]),unit='d')
            average_duration = average_duration/(nb_los+nb_win)
            print('Average Duration :', average_duration)
        except:
            print('No Average Duration available')
        try:
            print('Profit Factor : ',abs(round((sum(winners)/sum(loosers)),2)))
        except:
            print('Profit Factor : null')

        ########################
        ##### PLOT TRADES ######
        ########################

        plt.figure(figsize=(26,8),dpi=300)
        plt.title('Deep Learning '+ticker+' Trades')
        plt.plot(resultats['Open.S'],color='blue',lw=0.9)
        for i in range(0,len(trades)):
            if trades[i][2] == 1:
                plt.scatter(x=trades[i][1],y=trades[i][0],c='green',marker='^',lw=2.5)
            elif trades[i][2] == -1:
                plt.scatter(x=trades[i][1],y=trades[i][0],c='red',marker='^',lw=2.5)
            elif trades[i][2] == 2:
                plt.scatter(x=trades[i][1],y=trades[i][0],c='g',marker='x',lw=2.5)
            elif trades[i][2] == -2:
                plt.scatter(x=trades[i][1],y=trades[i][0],c='r',marker='x',lw=2.5)

        plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='g',marker='^',label='Open Long')
        plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='r',marker='^',label='Open Short')
        plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='g',marker='x',label='Close Long')
        plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='r',marker='x',label='Close Short')
        plt.legend()
        plt.savefig('backtest de '+str(ticker)+'_02.pdf')
        plt.close()

        #########################
        ##### PLOT E-CURVE ######
        #########################
        plt.figure(figsize=(26,8),dpi=300)
        plt.title('Deep Learning '+ticker+' Equity Curve')
        plt.plot(eq_curx,eq_cury)
        plt.scatter(x=eq_curx,y=eq_cury,c='orange',marker='o',lw=2.5)
        plt.savefig('backtest de '+str(ticker)+'_03.pdf')
        plt.close()


        f.close()
        sys.stdout = orig_stdout

        print('\r Boucle ', loop, 'terminée...',end='')
        print('Opération restantes :', len(constituents)-loop,'/',len(constituents),'\n')

        if nb_los == 0:
            nb_los = 0.0001
        if nb_win == 0:
            nb_win = 0.0001

        backtest = backtest.append([{'Symb.':ticker,'Name':name,'Sector':sector,'Period':duration,'Begin.':resultats.iloc[0,0],'End':resultats.iloc[-1,0],\
                          'Pnl $':pnl,'Pnl %':round(pnl/(10000/beta)*100,2),'Gross Profit':round(sum(winners),2),'Gross Loss':round(sum(loosers),2),'# Winners':nb_win,'# Loosers':nb_los,\
                          '% Winners':round((nb_win/(nb_win+nb_los) * 100),2),'MaxDD $':round(min(maxdd),2),'MaxDD %':round(min(maxdd)/(10000/beta)*100,2),'Aver. Win $':(round(sum(winners)/(nb_win))),\
                          'Aver. Loos $':(round(sum(loosers)/(nb_los))),'Aver. Duration':average_duration,\
                          'Total Trades':nbL+nbS,'% Long':round((nbL*100)/(nbL+nbS),2),'% Short':round((nbS*100)/(nbL+nbS),2), 'Profit Factor':abs(round((sum(winners)/sum(loosers)),2))}], ignore_index=True)


    # symbol / name / sector / period of backtest / debut / fin / pnl / total winners / total loosers / Nb winners / nb loosers / 
    # / % winners / Max DD / Aver Win / Aver Loos / Average duration / Profit Factor
        print(Fore.YELLOW,'Writing BACKTEST CSV AND SIGNALS.CSV',Style.RESET_ALL)
        backtest.to_csv('BackTest.csv')
        signals.to_csv('SIGNALS.csv')

        print(Fore.CYAN,'BACKTEST CSV WRITED AND SIGNALS.CSV',Style.RESET_ALL)

        
    #except:
     #   print(Fore.RED,Back.GREEN,'Il y a un problème en loop ',loop,', avec le ticker ',ticker,'absent de la base.',Style.RESET_ALL)
      #  continue
        
    tmps11=round(time.time()-tmps73,2)
    print(Fore.YELLOW,"Time for a time loop Signals Generation = %f" %tmps11,'seconds \n',Style.RESET_ALL)
    
    resultats = pd.DataFrame()
    resultats['Date'] = DDATE
    resultats['Move Up'] = MOVE_UP
    resultats['Confiance up'] = CONF_UP
    resultats['Move Down'] = MOVE_DWN
    resultats['Confiance Down'] = CONF_DWN
    resultats['Actual'] = ACTUAL
    resultats['Actual.S'] = ACTUAL_S
    resultats['Open.S'] = OPEN_S
    resultats['DMP_CP'] = DMP_CP
    
    signals = pd.DataFrame()
    signals['Date'] = DATE
    signals['Ticker'] = TICKER
    signals['Open Long'] = GREEN
    signals['Close Long'] = GREEN_CLOSE
    signals['Open Short'] = RED
    signals['Close Short'] = RED_CLOSE 
    signals['Close Price'] = PRICE
    
    try:

        resultats = pd.read_csv('RESULTS/'+LeNom+'RESULTATS_'+stamp+'.csv')
        try:
            resultats = resultats.drop('Unnamed: 0',axis=1)
        except:
            pass
        print('Le fichier RESULTATS_'+stamp+'.csv existe déjà')


    except:
        print('\n Création du fichier RESULTATS_'+stamp+'.csv')
        try:
            resultats = resultats.drop('Unnamed: 0',axis=1)
        except:
            pass
        resultats.to_csv('RESULTS/'+LeNom+'RESULTATS_'+stamp+'.csv')

    try:

        signals = pd.read_csv('RESULTS/'+LeNom+'SIGNALS_'+stamp+'.csv')
        try:
            signals = signals.drop('Unnamed: 0',axis=1)
        except:
            pass
        print('Le fichier SIGNALS_'+stamp+'.csv existe déjà')


    except:
        print('\n Création du fichier SIGNALS_'+stamp+'.csv')
        try:
            signals = signals.drop('Unnamed: 0',axis=1)
        except:
            pass
        signals.to_csv('RESULTS/'+LeNom+'SIGNALS_'+stamp+'.csv')
    
    
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Signals Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)        

AttributeError: 'numpy.int64' object has no attribute 'listdir'

In [247]:
len(resultats),len(DDATE),len(MOVE_UP),len(CONF_UP),len(MOVE_DWN),len(CONF_DWN),len(DMP_CP),len(ACTUAL),len(ACTUAL_S),len(OPEN_S)

(154, 215, 215, 215, 215, 215, 215, 215, 215, 215)

In [248]:
len(DATE),len(TICKER),len(GREEN),len(GREEN_CLOSE),len(RED),len(RED_CLOSE),len(PRICE)

(215, 215, 215, 215, 215, 215, 215)

In [249]:
resultats.head()

Date  Move Up  Confiance up  Move Down  Confiance Down  \
Date                                                                       
2017-05-26  2017-05-26        0      0.070655          0        0.005105   
2017-05-30  2017-05-30        0     35.327808          0        0.000218   
2017-05-31  2017-05-31        0      0.001335          0       13.964126   
2017-06-01  2017-06-01        0      0.002837          0        0.804174   
2017-06-02  2017-06-02        0      0.000125          0       46.110298   

                Actual    Actual.S      Open.S     DMP_CP  
Date                                                       
2017-05-26  119.087616  119.544785  129.350006  86.522957  
2017-05-30  118.591072  119.544785  129.350006  99.998764  
2017-05-31  119.289909  119.544785  130.289993 -99.980881  
2017-06-01  119.528999  119.544785  130.380005 -99.296867  
2017-06-02  120.126671  119.544785  130.309998 -99.999451

In [256]:
resultats.tail()

Date  Move Up  Confiance up  Move Down  Confiance Down  \
Date                                                                       
2017-12-28  2017-12-28        0      5.105782          0        0.004911   
2017-12-29  2017-12-29        0      1.264334          0        1.234016   
2018-01-02  2018-01-02        0      0.000632          1       78.861580   
2018-01-03  2018-01-03        1     52.266758          0        0.000003   
2018-01-04  2018-01-04        0      0.001117          0        0.411175   

                Actual    Actual.S      Open.S     DMP_CP  
Date                                                       
2017-12-28  113.183418  113.754372  120.599998  99.807800  
2017-12-29  113.588219  113.754372  120.809998   1.213517  
2018-01-02  112.091393  111.130112  119.489998 -99.998398  
2018-01-03  110.218033  111.130112  117.669998  99.999985  
2018-01-04  111.488907  111.130112         NaN -99.458313

In [250]:
signals.head()

Date Ticker  Open Long  Close Long  Open Short  Close Short  \
0  2018-01-03    LHX          0           0           0            0   
1  2018-01-03     MS          0           0           0            0   
2  2018-01-03    DHI          0           0           0            0   
3  2018-01-03   ABMD          0           0           0            0   
4  2018-01-03    MCK          0           0           0            0   

   Close Price  
0   138.889786  
1    49.750721  
2    51.647675  
3   195.820007  
4   153.679352

In [251]:
signals['Open Long'].sum(),signals['Open Short'].sum() # Nombre de signaux d'ouvertures

(5, 13)

In [253]:
signals['Close Long'].sum(),signals['Close Short'].sum() # Nombre de signaux de fermeture

(13, 8)

In [252]:
signals.shape

(270, 7)

In [255]:
prepared_data = df # On reserve les données préparées aucas où voilà quoi

In [257]:
df = signals
choupinet = pd.DataFrame()

DT = []
OL = []
CL = []
OS = []
CS = []
XP = []
PNL = []
CUM = []
YMCA = []
NETL = []
NETS = []
LAT = []
XPL = []
XPS = []
MDD = []
NXP = []

ol = 0
cl = 0
os = 0
cs = 0
xp = 0
pnl = 0
cum = 0
ymca = 0
netl = 0
nets = 0
lat = 0
xpl = 0
xps = 0
mdd = 0
nxp = 0

df.head(1)

Date Ticker  Open Long  Close Long  Open Short  Close Short  \
0  2018-01-03    LHX          0           0           0            0   

   Close Price  
0   138.889786

In [264]:
for i in range(0,df.shape[0]-1):
    if df.iloc[i+1,0] == df.iloc[i,0]:
        ol += df.iloc[i,2]
        cl += df.iloc[i,3]
        os += df.iloc[i,4]
        cs += df.iloc[i,5]
        if df.iloc[i,3] == 1 or df.iloc[i,5] == 1 :
            xp = xp - (df.iloc[i,6] * df.iloc[i,7])
        if df.iloc[i,2] == 1 or df.iloc[i,4] == 1 :
            xp = xp + (df.iloc[i,6] * df.iloc[i,7])
        
        try:
            cum += df.iloc[i,8]
        except:
            cum += 0
        pnl += df.iloc[i,8]
        lat += df.iloc[i,10]
        
        if df.iloc[i,2] == 1:
            xpl = xpl + (df.iloc[i,6] * df.iloc[i,7])
        if df.iloc[i,3] == 1:
            xpl = xpl - (df.iloc[i,6] * df.iloc[i,7])
        
        if df.iloc[i,4] == 1:
            xps = xps  -(df.iloc[i,6] * df.iloc[i,7])
        if df.iloc[i,5] == 1:
            xps = xps + (df.iloc[i,6] * df.iloc[i,7])
        
        
        nxp =xpl+  xps
        
    else:
        
        ymca = 400000 + cum +lat
        netl = netl + ol - cl
        nets = nets + os - cs
        DT.append(df.iloc[i-1,0])
        OL.append(ol)
        CL.append(cl)
        OS.append(os)
        CS.append(cs)
        XP.append(xp)
        PNL.append(pnl)
        CUM.append(cum)
        YMCA.append(ymca)
        NETL.append(netl)
        NETS.append(nets)
        LAT.append(lat)
        if min(LAT) < 0:
             mdd = min(LAT)
        XPL.append(xpl)
        XPS.append(xps)
        MDD.append(mdd)
        NXP.append(nxp)
        ol = 0
        cl = 0
        os = 0
        cs = 0
        pnl = 0
        lat = 0
        mdd = 0

choupinet['Date'] = DT
choupinet['Open Long'] = OL
choupinet['Close Long'] = CL
choupinet['Open Short'] = OS
choupinet['Close Short'] = CS
choupinet['Exposure Long'] = XPL
choupinet['Exposure Short'] = XPS
choupinet['Exposure'] = XP
choupinet['Net Exposure'] = NXP
choupinet['PNL'] = PNL
choupinet['Unrealized PNL'] = LAT
choupinet['Cumulative PNL'] = CUM
choupinet['NAV'] = YMCA
choupinet['Net Long'] = NETL
choupinet['Net Short'] = NETS
choupinet['Maximum Drawdown'] = MDD

IndexError: single positional indexer is out-of-bounds

In [ ]:
plt.figure(figsize=(26,8),dpi=300)
plt.title('Equity curve 2019')
plt.ylabel('NAV')
plt.xlabel('Days')
plt.plot(choupinet['NAV'],color='purple',lw=0.9)
plt.savefig('nav_2019.png')

choupinet.tail()

In [263]:
choupinet_le_canard_il_est_un_vrai_cafard

'ADJUSTED/DL_2018-12-31/'

In [ ]:
choupinet.to_csv(choupinet_le_canard_il_est_un_vrai_cafard+'portfolio_2018.csv')

In [268]:
LeNom

'verif_ADJUSTED/'